In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import json
import os
import math
import librosa

DATASET_PATH = "drive/MyDrive/Music Genre Classifier Dataset/genres_original"
JSON_PATH = "NN_Features.json"
SAMPLE_RATE = 22050 # Hz, number of samples/sec
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

def save_mfccs(dataset_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """
    dataset_path = path to dataset
    output_path = path to output json file
    num_mfccs = number of coefficients in MFCC features
    n_fft = length of each window for fast fourier transform
    hop_length = hop length between each overlapping window
    num_segments = number of segments to divide the 30 second audio input into
    """
    output = {
        "mapping": [], # Text mapping eg. classical
        "mfcc": [], # MFCC features for each segment,
        "label": [] # numerical labels to refer to each text mapping
    }

    samples_per_segment = int(SAMPLES_PER_TRACK/ num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length) # Output per segment num_mfcc * num_windows
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # Each iteration we are going through each genre
        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:
            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("\\")[-1]
            output["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))
            for audio_files in filenames:
                print(audio_files)
            # load audio file
                file_path = os.path.join(dirpath, audio_files)  # Get file path
                file_path = str(file_path.replace("\\", "/"))
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mfcc
                    mfcc = librosa.feature.mfcc(y = signal[start:finish], sr = sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        output["mfcc"].append(mfcc.tolist())
                        output["label"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))
    return output


In [15]:
output = save_mfccs(DATASET_PATH)

Streaming output truncated to the last 5000 lines.
classical.00066.wav
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00066.wav, segment:1
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00066.wav, segment:2
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00066.wav, segment:3
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00066.wav, segment:4
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00066.wav, segment:5
classical.00068.wav
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00068.wav, segment:1
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00068.wav, segment:2
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00068.wav, segment:3
drive/MyDrive/Music Genre Classifier Dataset/genres_original/classical/classical.00068.wav, s

In [16]:
with open(JSON_PATH, "w") as fp:
    json.dump(output, fp, indent=4)